In [1]:
"""You are a sophisticated python data scientist/analyst.
You are provided with a question and a dataset.
Generate a python script to be run in a Jupyter notebook that calculates the result and renders a plot.
Only one code block is allowed, use markdown code blocks.
Install additional packages (using !pip syntax) before importing them.

The following libraries are already installed:
- jupyter
- numpy
- pandas
- matplotlib
- seaborn
- plotly (not supported yet)"""

'You are a sophisticated python data scientist/analyst.\nYou are provided with a question and a dataset.\nGenerate a python script to be run in a Jupyter notebook that calculates the result and renders a plot.\nOnly one code block is allowed, use markdown code blocks.\nInstall additional packages (using !pip syntax) before importing them.\n\nThe following libraries are already installed:\n- jupyter\n- numpy\n- pandas\n- matplotlib\n- seaborn\n- plotly (not supported yet)'

In [1]:
import e2b
from langchain_groq import ChatGroq
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from dotenv import load_dotenv
from e2b_code_interpreter import Sandbox
from langchain_core.tools import tool
import pandas as pd
import sys
import base64
import os

load_dotenv()

True

In [2]:
llm = ChatNVIDIA(
    model="nv-mistralai/mistral-nemo-12b-instruct",
    temperature=0.0,
    api_key=os.getenv('NVIDIA_API_KEY'),
)

In [3]:
def summarizer(file_path):

    df = pd.read_csv(file_path)

    buffer = []
    buffer.append("## Dataset Overview")
    buffer.append(f"- Shape: {df.shape} (rows × columns)")
    
    dtype_counts = df.dtypes.value_counts().to_dict()
    buffer.append("\n## Data Types")
    for dtype, count in dtype_counts.items():
        buffer.append(f"- {dtype}: {count} columns")
    
    buffer.append("\n## Descriptive Statistics")
    buffer.append(df.describe(include='all').to_markdown())
    
    missing = df.isna().sum()
    buffer.append("\n## Missing Values")
    buffer.append(missing[missing > 0].to_markdown() if missing.any() else "No missing values found")
    
    buffer.append("\n## Unique Value Counts")
    for col in df.select_dtypes(include=['object', 'category']):
        buffer.append(f"- {col}: {df[col].nunique()} unique values")
    
    buffer.append("\n## Data Sample")
    buffer.append(df.head(3).to_markdown())
    
    return "\n".join(buffer)


In [4]:

SUMMARIZER_PROMPT = """You are a summarizer assistant, who is working in a team.
        your task is to summarize the given csv data in structured format so that other agent can use it effectively for analysis.
        provide dataset name by yourself based on following data. your summary should be useful for further data analysis tasks.
        IMPORTANT INSTRUCTION: DO NOT CHANGE THE ATTRIBUTE NAME, YOU SHOULD KEEP AS IT IS, EITHER IT IS LOWER OR IN UPPER CASE.
        Here is the {df}
        Provide a comprehensive summary of the dataset:
         
"""

def build_summarizer_prompt(file_path):
    prompt = SUMMARIZER_PROMPT.format(
        df=summarizer(file_path)
    )
    return prompt


def summarizer_agent(llm, summarizer_system_prompt):
    print(f' {"="*50} Analyzing the data {"="*50}')
    messages = [
            {'role': 'system', 'content': "You are a helpful data analysis assistant."},
            {'role': 'user', 'content': summarizer_system_prompt}
        ]
    response = llm.invoke(messages)
    print(f'SUMMARY OF THE DATA: {response.content}')
    return response

In [5]:
from pydantic import BaseModel
from typing import List

class StructuredPlan(BaseModel):
    key_observations: str
    plan: str
    recommendations: str
    attributes: List[str]

In [6]:
DESCRIPTIVE_PLANNER_PROMPT = """
You are a data scientist specializing in spend analytics.

Your task is to create a structured, step-by-step **analysis plan** to address the following query:
**User Query:** {query}

---

**Data Summary:**
{df_summary}

---

### INSTRUCTIONS - MUST FOLLOW STRICTLY:
- Use the data summary and user query to create a practical, executable analysis plan.
- DO NOT provide Any code. But your plan must be executable in a Jupyter notebook using a python code by another agent.
- ⚠️ DO NOT rename or change attribute (column) names. Use them exactly as shown in the Data Summary.
- ✅ Use column names in lowercase or uppercase only as per the original dataset.
- ❌ Do not invent new columns. Only use columns that exist.
- ⚠️ Do not suggest prediction, classification, or machine learning. Only descriptive statistics.
- 🧠 Be action-oriented and practical in the analysis plan.

---

### Your response must strictly follow the below structure:

1. **Key Observations**
   - Bullet point summary of 3-5 insights from the data summary.
   - Focus on data types, missing values, uniqueness, and distributions.

2. **Step-by-Step Analysis Plan**
   - Bullet points only.
   - Each step must use **only existing attributes** from the dataset.
   - Provide 1 or more steps to answer the user query.
   - Try to solve as simple as possible.

3. **Recommended output analysis**
   - Suggest only one of: `Table`, `Bar Chart`, `Pie Chart`, or `Text Summary` per step.
   - Match format to the step it supports best.

4. **Attributes to be Used**
   - List all attributes (column names) used in the steps above.
   - Use their exact name (case-sensitive) as given in the Data Summary.
   - DO NOT modify, rename, or infer attributes.

"""


def build_descriptive_planner_prompt(query, df_summary):
    prompt = DESCRIPTIVE_PLANNER_PROMPT.format(
        df_summary=df_summary,
        query=query,
    )
    return prompt

def Descriptive_planner_agent(llm, descriptive_planner_prompt):
    print(f" {'='*50} Making plan... {'='*50}")
    messages = [
        {'role': 'system', 'content': 'you are helpful data analysis assistant'},
        {'role': 'user', 'content': descriptive_planner_prompt}
    ]
    structured_llm = llm.with_structured_output(StructuredPlan)
    response = structured_llm.invoke(messages)
    return response

In [7]:
class PythonCode(BaseModel):
    python_code: str

DESCRIPTIVE_CODER_PROMPT = """
You are a Python data scientist who writes advanced, production-ready code to solve data analysis problems **strictly** based on a plan.
---
### Your task:
Write Python code based on the following inputs:

- `dataset_path`: Full path to the dataset available in the sandbox → {dataset_path}
- `descriptive_plan`: Step-by-step analysis logic → {descriptive_plan}
- `attributes`: Required columns in the dataset → {descriptive_attributes}
- `recommendations`: Outputs required (tables, text, or charts) → {descriptive_recommendations}
---
### 🔐 Sandbox Rules & Constraints:
- **Use only columns explicitly listed** in `attributes`. Never assume column names.
- Use print() statements for intermediate debug output or summaries.
- Avoid user interaction functions (e.g., input() or interactive widgets).
- Make sure write code without indentation errors.
- **display visualizations using matplotlib/plotly/seaborn or any other visualization library directly in the notebook. can generate any number of charts in one run time. don't worry about saving the visualizations to a file.**
- you have access to the internet and can make api requests.
- you also have access to the filesystem and can read/write files.
- you can install any pip package (if it exists) if you need to be running `!pip install`. The usual packages for data analysis are already preinstalled though.
🛠️ Code Requirements:
- Use modular, well-commented code. Functions are preferred.
- Make sure there is no syntax error in the code you generate.
- Import all the necessary libraries at the beginning of the code block.
- Code must be fully sandbox-executable, headless, and debuggable.
- You can write code that creates visualizations and structured textual or tabular outputs.
📦 Output Format (strict):
### Your response must strictly follow this format:
```python
# your Python code here
```
"""

def build_descriptive_coder_prompt(dataset_path, descriptive_plan, descriptive_recommendations, descriptive_attributes):
    prompt = DESCRIPTIVE_CODER_PROMPT.format(
        dataset_path=dataset_path,
        descriptive_attributes=descriptive_attributes,
        descriptive_plan=descriptive_plan,
        descriptive_recommendations=descriptive_recommendations
    )
    return prompt

def descriptive_coder_agent(llm, descriptive_coder_prompt):
    messages = [
        {'role': 'system', 'content': 'you are helpful python coding assistant'},
        {'role': 'user', 'content': descriptive_coder_prompt}
    ]
    structured_llm = llm.with_structured_output(PythonCode)
    response = structured_llm.invoke(messages)
    return response


In [8]:
CODE_VALIDATOR_PROMPT = """Your task is to review the provided python code.
        Check for syntax errors, logical errors, Indentation errors and adherence to best practices.
        If the code is correct, return it as is. else correct the code and return it.
        Here is the code:\n {code}.

###Your response must strictly follow this format:
```python
#your code goes here
```
"""

def build_code_validator_prompt(code):
    prompt = CODE_VALIDATOR_PROMPT.format(
        code=code
    )
    return prompt

def code_validator_agent(llm, code_validator_prompt):
    messages = [
        {'role': 'system', 'content': 'you are python code reviewer'},
        {'role': 'user', 'content': code_validator_prompt}
    ]
    response = llm.invoke(messages)
    return response

In [9]:
sbx = Sandbox(timeout=1000)
running_sandboxes = sbx.list()
sandbox_id = running_sandboxes[0].sandbox_id
sbx = Sandbox.connect(sandbox_id)

In [10]:
def upload_data(file_path):
    print(f'Uploading data from {file_path} to the sandbox...')
    dataset_path = file_path

    if not os.path.exists(dataset_path):
        raise FileNotFoundError("Dataset file not found")

    try:
        with open(dataset_path, "rb") as f:
            dataset_path_in_sandbox = sbx.files.write(os.path.basename(dataset_path), f)
        print(f'Dataset uploaded to {dataset_path_in_sandbox}')
        return dataset_path_in_sandbox
    except Exception as error:
        print("Error during file upload:", error)
        raise error


In [11]:
import base64
import re
from langchain_core.tools import tool

def execute_code(code: str) -> str:
    """
    Executes Python code in a sandbox environment.
    """
    print('Executing code in sandbox...')

    execution = sbx.run_code(code)
    print('Code execution finished!')

    if execution.error:
        print('AI-generated code had an error.')
        print(execution.error.name)
        print(execution.error.value)
        print(execution.error.traceback)

        error_content = {
            'error': execution.error.name,
            'message': execution.error.value,
            'code': code
        }
        return str(error_content)
    
    output = execution.logs.stdout if execution.logs and execution.logs.stdout else execution.text
    output = ''.join(output) if isinstance(output, list) else output

    print(output)
    visual_idx = 0
    for result in execution.results:
        if result.png:
            with open(f'visual-{visual_idx}.png', 'wb') as f:
                f.write(base64.b64decode(result.png))
            print(f'Visual saved to visual-{visual_idx}.png')
            visual_idx += 1

        if result.chart:
            chart = result.chart
            with open('Chart_data.txt', 'w') as f:
                f.write(f"Type: {chart.type}\n")
                f.write(f"Title: {chart.title}\n")
                f.write(f"X Label: {chart.x_label}\n")
                f.write(f"Y Label: {chart.y_label}\n")
                f.write(f"X Unit: {chart.x_unit}\n")
                f.write(f"Y Unit: {chart.y_unit}\n")
                f.write("Elements:\n")
                for element in chart.elements:
                    f.write("\n")
                    for attr in ['label', 'value', 'x', 'y', 'group']:
                        if hasattr(element, attr):
                            f.write(f"  {attr.capitalize()}: {getattr(element, attr)}\n")


    return output
    

In [12]:
FINAL_RESPONSE_PROMPT = """
You are acting as the final response generator in an intelligent data analysis pipeline — the Finisher.
Your role is to synthesize and articulate a clear, confident, and conversational summary of the analysis based on the following inputs:

🔹 **Key Observations:** {key_observations}
🔹 **Plan of Action:** {plan}
🔹 **Execution Results:** {execution_results}

Your job is to present the *final summary* as if you conducted the entire analysis yourself — confidently and professionally.

### Instructions:
- Frame the response as a concluding, insightful summary.
- Start conversationally, but grounded in data — *“Here's a quick summary of your data…”*
- If `execution_results` is empty, missing, or an error occurred, fall back to the plan and key observations to still generate a confident output.
- Clearly highlight important patterns, spend trends, or anomalies (if present).
- Offer value-driven recommendations or next steps if applicable.
- Do **not** say “I dont know” or mention failures — always provide a best-effort insight based on available context.

The goal is to make the user feel like they've received expert-level, high-quality insights.
"""


def build_final_response_prompt(key_observations, plan, execution_results):
    prompt = FINAL_RESPONSE_PROMPT.format(
        key_observations=key_observations,
        plan=plan,
        execution_results=execution_results
    )
    return prompt

def final_response_generator(llm, final_message):
    messages = [
        {'role': 'system', 'content': "You are a helpful summarizer assistant."},
        {'role': 'user', 'content': final_message}
    ]
    response = llm.invoke(messages)
    return response

In [13]:
def main(user_query):
    file_path = r'C:\Users\arun5\Desktop\Spend_analyzer\src\tests\transaction_data_250.csv'
    summarizer_system_prompt = build_summarizer_prompt(file_path)
    df_summary = summarizer_agent(llm, summarizer_system_prompt)
    descriptive_planner_prompt = build_descriptive_planner_prompt(user_query, df_summary)
    descriptive_response = Descriptive_planner_agent(llm, descriptive_planner_prompt)
    print(f" {descriptive_response.plan}")
    dataset_path = upload_data(r'C:\Users\arun5\Desktop\Spend_analyzer\src\tests\transaction_data_250.csv')
    descriptive_coder_prompt = build_descriptive_coder_prompt(
        dataset_path=dataset_path.path,
        descriptive_plan=descriptive_response.plan,
        descriptive_recommendations=descriptive_response.recommendations,
        descriptive_attributes=descriptive_response.attributes,
    )

    descriptive_code = descriptive_coder_agent(llm, descriptive_coder_prompt)
    #print(descriptive_code.python_code)
    code_validator_prompt = build_code_validator_prompt(descriptive_code.python_code)
    valid_code = code_validator_agent(llm, code_validator_prompt)

    pattern = re.compile(
        r"```python\n(.*?)\n```", re.DOTALL
    )

    def match_code_blocks(valid_code):
        match = pattern.search(valid_code.content)
        if match:
            code = match.group(1)
            print(code)
            return code
        return ""

    print(f" {'='*30} CODE BLOCK {'='*30} \n {valid_code.content} \n {'='*30} END OF CODE BLOCK {'='*30}")
    executed_results = execute_code(match_code_blocks(valid_code))
    final_message = build_final_response_prompt(
        key_observations=descriptive_response.key_observations,
        plan=descriptive_response.plan,
        execution_results=executed_results
    )
    final_response = final_response_generator(llm, final_message)
    print(final_response.content)

In [19]:
if __name__ == "__main__":
    while True:
        user_query = input('Ask your query: ')
        if user_query.lower() in ['exit', 'quit']:
            print("Exiting the program.")
            break
        main(user_query)

 ================================================== Analyzing the data ==================================================
SUMMARY OF THE DATA: **Dataset Name:** Corporate Expenses and Contracts

**Summary:**

- **Size:** 250 rows × 11 columns
- **Data Types:** 9 object columns, 2 int64 columns
- **Key Attributes:** transaction_id (unique identifier), date, category, sub_category, vendor, amount, contract_type, budget, payment_terms, region, notes
- **Missing Values:** 56 missing values in the 'region' column
- **Unique Values:**
  - transaction_id: 250
  - date: 146
  - category: 6 (IT, Marketing, Travel, HR, Legal, Other)
  - sub_category: 19
  - vendor: 52
  - contract_type: 3 (Contract, Spot, Other)
  - payment_terms: 4 (Net 15, Net 30, Net 60, Other)
  - region: 3 (APAC, EMEA, Global)
  - notes: 12
- **Descriptive Statistics:**
  - **Amount:** Mean = $131,563, Median = $125,726, Standard Deviation = $64,084, Min = $23,729, Max = $248,773
  - **Budget:** Mean = $121,068, Median = $1

AttributeError: 'SuperChart' object has no attribute 'x_label'